# 0. 準備

## 0. 準備の準備

本ハンズオンで利用するデータセットの一部はGit LFSを介して提供されている。
Git LFSをインストールせよ。


### 回答

Dev Containerを利用し、Dockerのイメージをビルドするタイミングで`sudo apt install git-lfs -y`コマンドでインストールするようにした。
参考：[.devcontainer/postCreateCommand.sh](../.devcontainer/postCreateCommand.sh)


## 1. データセットの入手

本ハンズオンではAmazonのShopping Queries Datasetを利用する。
以下から `git clone` せよ。

https://github.com/amazon-science/esci-data


### 回答

リポジトリ直下（ir100ディレクトリ）で以下のコマンドを実行。DockerのVolumeとしてマウントしてあるので、ホスト側のディレクトリにはデータは存在しない。


In [1]:
!git clone https://github.com/amazon-science/esci-data ../esci-data

fatal: destination path '../esci-data' already exists and is not an empty directory.


## 2. ライブラリの入手

このデータセットの一部はParquet形式である。
Parquet形式のファイルを読み込むためのライブラリを入手せよ。


### 回答

Poetryをパッケージマネージャーとして利用するので、以下のコマンドで追加。


In [2]:
!poetry add pandas datasets

The following packages are already present in the pyproject.toml and will be skipped:

  • pandas
  • datasets

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


## 3. 製品データの読み込み

ライブラリの機能で製品データ（以下）をメモリに読み込め。

```
./esci-data/shopping_queries_dataset/shopping_queries_dataset_products.parquet
```

（想定実行時間：1分以内）



### 回答


In [3]:

import pandas as pd
import pathlib

esci_path = pathlib.Path("../esci-data/shopping_queries_dataset")
print("Start loading products...")
df_products = pd.read_parquet(esci_path.joinpath("shopping_queries_dataset_products.parquet"))
print("Loaded products.")

Start loading products...
Loaded products.


## 4. 製品データの確認

製品を1つランダムに取り出し表示せよ。
何度か実行せよ。


### 回答



In [4]:
df_products.sample()

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale
1728954,B09F95G7RN,Without You (8-Bit Miley Cyrus & The Kid Laroi...,None,None,None,None,us


In [5]:

df_products.sample()

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale
1057506,B009HOPXD4,日本電興(NIHON DENKO) 電磁式エアーポンプ 30L NIP-30L ホワイト,None,定格電圧:100V(50/60Hz)\n消費電力:33W\n定格圧力:0.012MPA(0....,日本電興,ホワイト,jp


In [6]:

df_products.sample()

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale
1707016,B07T24GKWC,"Sorbus Planter Pots and Tray Caddy, 3 Buckets ...",Beautifully display small succulents with the ...,PLANTER POT CADDY SET — Beautifully display su...,Sorbus,Silver,us


## 5. クエリデータの読み込み

クエリデータ（以下）をメモリに読み込め。

```
./esci-data/shopping_queries_dataset/shopping_queries_dataset_examples.parquet
```

（想定実行時間：1分以内）



### 回答




In [7]:
print("Start loading examples...")
df_examples = pd.read_parquet(esci_path.joinpath("shopping_queries_dataset_examples.parquet"))
print("Loaded examples.")

Start loading examples...
Loaded examples.


## 6. クエリデータの確認

クエリを1つランダムに取り出し表示せよ。
何度か実行せよ。



### 回答

In [8]:
df_examples.sample()

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split
587601,587601,cordones blancos sin nudo,29055,B078MF9X9J,es,E,1,1,train


In [9]:
df_examples.sample()

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split
445435,445435,calmante de the body shop,21663,B07GSMQ4NL,es,C,1,1,test


In [10]:
df_examples.sample()

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split
1022100,1022100,hover soccer ball indoor,51564,B07TW2FYNV,us,E,1,1,test


## 7. フィルタリング

題材の製品データ、クエリデータそれぞれ `product_locale` が `us` であるものを抽出せよ。

これ以降、特に断りがなければ、以下の理由からこれらのサブセットを用いる。

- 単語がスペース区切りで扱いやすいため。
- `product_locale` 内では `product_id` がユニークで扱いやすいため。

また、それぞれ**英語製品データ**、**英語クエリデータ**と呼ぶ。


### 回答


In [11]:
df_products_us = df_products[df_products["product_locale"] == "us"]
df_examples_us = df_examples[df_examples["product_locale"] == "us"]

## 8. 結合

英語製品データと英語クエリデータを、`product_id` をキーとして結合せよ。
これを**英語結合データ**と呼ぶ。

また、英語結合データのエントリを1つランダムに取り出し表示せよ。
何度か実行せよ。


### 回答


In [12]:
pd_examples_products_us = pd.merge(
    df_examples_us,
    df_products_us,
    how='left',
    left_on=['product_id'],
    right_on=['product_id']
)

pd_examples_products_us.sample()


,example_id,query,query_id,product_id,product_locale_x,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale_y
892314,1121667,juvenatur antiarrugas,56750,B00U2VQYWK,us,E,1,1,test,RoC Retinol Correxion Max Wrinkle Anti-Aging S...,None,HOLIDAY GIFT: Give the gift of self care and b...,RoC,White,us


In [13]:

pd_examples_products_us.sample()

,example_id,query,query_id,product_id,product_locale_x,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale_y
38618,45612,10x10x6 cake box without window,1635,B08G8XV4NH,us,I,1,1,test,[10 Pack of Each] 12 Inch Cake Boxes with Cake...,<b>You Can Have Your Cake And Eat It Too! Get ...,✅EASY TO ASSEMBLE: We have taken out the frust...,Dineton,None,us


In [14]:

pd_examples_products_us.sample()

,example_id,query,query_id,product_id,product_locale_x,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale_y
390185,504320,champion high tops,24694,B07LGB9L6G,us,E,1,1,train,Champion Kids Rally Hype Mid (Big Kid) Scarlet...,None,None,Champion,None,us
